# Plotting an example showing that setting correctly the acceleration parameters is a hard task

## This code imports csv results generated by the `benchmarks.py` (which does not yet produces the desired plot):
- *ridge_sketch/benchmark_results/kernel_small_ac/residual_norms.csv*
- *ridge_sketch/benchmark_results/kernel_small_ac/times.csv*

In [127]:
import os
import numpy as np
import pandas as pd 
from scipy.sparse import random as sprandom
from timeit import default_timer
import time
import re
import plotly.express as px 
import plotly.graph_objs as go
from PIL import ImageColor
import functools
import matplotlib.pyplot as plt

from kernel_ridge_sketch import KernelRidgeSketch

In [128]:
COLORS = px.colors.qualitative.Plotly
# blue, red, green, purple, cyan, pink, ...

LINE_STYLES = ["solid", "dot", "dash", "longdash", "dashdot", "longdashdot"]

SYMBOLS = [
    "circle",
    "square",
    "star",
    "x",
    "triangle-up",
    "pentagon",
    "cross",
]

# with acceleration ($\mu = $0.8, $\nu = $1.0)
# with acceleration ($\mu = $0.8, $\nu = $1.2)
# with acceleration ($\mu = $0.9, $\nu = $1.0)
# 'with acceleration ($\\mu = $0.8, $\\nu = $1.0)'
# $\text{acceleration }(\mu = 0.8, \nu = 1.0)$


COLOR_DICT = {
    "no acceleration": COLORS[0],
    "acceleration ($\\mu = $0.8, $\\nu = $1.2)": COLORS[1],
    "acceleration ($\\mu = $0.8, $\\nu = $1.0)": COLORS[2],
    "acceleration ($\\mu = $0.9, $\\nu = $1.0)": COLORS[3],
    "$\\text{no acceleration}$": COLORS[0],
    "$\\text{acceleration }(\\mu = 0.8, \\nu = 1.2)$": COLORS[1],
    "$\\text{acceleration }(\\mu = 0.8, \\nu = 1.0)$": COLORS[2],
    "$\\text{acceleration }(\\mu = 0.9, \\nu = 1.0)$": COLORS[3],
    # "acceleration $(\\mu = 0.8, \\nu = 1.0)$": COLORS[2],
}

LINE_DICT = {
    "no acceleration": LINE_STYLES[0],
    "acceleration ($\\mu = $0.8, $\\nu = $1.2)": LINE_STYLES[1],
    "acceleration ($\\mu = $0.8, $\\nu = $1.0)": LINE_STYLES[2],
    "acceleration ($\\mu = $0.9, $\\nu = $1.0)": LINE_STYLES[3],
    "$\\text{no acceleration}$": LINE_STYLES[0],
    "$\\text{acceleration }(\\mu = 0.8, \\nu = 1.2)$": LINE_STYLES[1],
    "$\\text{acceleration }(\\mu = 0.8, \\nu = 1.0)$": LINE_STYLES[2],
    "$\\text{acceleration }(\\mu = 0.9, \\nu = 1.0)$": LINE_STYLES[3],
    # "acceleration $(\\mu = 0.8, \\nu = 1.0)$": LINE_STYLES[2],
}

SYMBOL_DICT = {
    "no acceleration": SYMBOLS[0],
    "acceleration ($\\mu = $0.8, $\\nu = $1.2)": SYMBOLS[1],
    "acceleration ($\\mu = $0.8, $\\nu = $1.0)": SYMBOLS[2],
    "acceleration ($\\mu = $0.9, $\\nu = $1.0)": SYMBOLS[3],
    "$\\text{no acceleration}$": SYMBOLS[0],
    "$\\text{acceleration }(\\mu = 0.8, \\nu = 1.2)$": SYMBOLS[1],
    "$\\text{acceleration }(\\mu = 0.8, \\nu = 1.0)$": SYMBOLS[2],
    "$\\text{acceleration }(\\mu = 0.9, \\nu = 1.0)$": SYMBOLS[3],
    # "acceleration $(\\mu = 0.8, \\nu = 1.0)$": SYMBOLS[2],
}

In [129]:
def apply_style(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        fig = func(*args, **kwargs)
        fig.update_layout(
            template="plotly_white", font=dict(size=20,),
        )
        return fig

    return wrapper

## Choose data

### Artificial problem

In [130]:
# Configs:
#  {'name': 'kernel_small_ac', 'tolerance': 0.0001, 'max_iterations': 10000, 'alpha': 0.00025, 'problems': ('primal_random',), 'density': 0.25, 'operator_modes': (False,), 'algo_modes': ('auto', 'accel'), 'accel_params': ((0.9, 1.1), (0.8, 1.0), (0.8, 1.2)), 'solvers': {'subsample'}, 'sparse_formats': ('csc',), 'kernel': 'Matern', 'kernel_parameters': (1.0, 0.5), 'smaller_dimension': 2000, 'larger_dimension': 4000, 'sketch_sizes': (63,)} 

# Acceleration parameters: ((0.9, 1.1), (0.8, 1.1), (0.9, 1.2))
residuals_df = pd.read_csv("/home/nidham/phd/RidgeSketch/ridge_sketch/benchmark_results/accel_exp/residual_norms.csv", index_col=0)

In [131]:
residuals_df

,primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.9-1.0) | residual_norms (median),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.9-1.0) | residual_norms (1st quartile),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.9-1.0) | residual_norms (3rd quartile),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.0) | residual_norms (median),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.0) | residual_norms (1st quartile),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.0) | residual_norms (3rd quartile),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.2) | residual_norms (median),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.2) | residual_norms (1st quartile),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.2) | residual_norms (3rd quartile),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = auto | accel_param = (0.0-0.0) | residual_norms (median),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = auto | accel_param = (0.0-0.0) | residual_norms (1st quartile),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = auto | accel_param = (0.0-0.0) | residual_norms (3rd quartile)
0,1.000000,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.960681,0.960681,0.960681,9.606807e-01,9.606807e-01,9.606807e-01,0.960681,0.960681,0.960681,0.960681,0.960681,0.960681
2,0.918090,0.918090,0.918090,9.181836e-01,9.181836e-01,9.181836e-01,0.918067,0.918067,0.918067,0.918064,0.918064,0.918064
3,0.880549,0.880549,0.880549,8.807582e-01,8.807582e-01,8.807582e-01,0.880498,0.880498,0.880498,0.880490,0.880490,0.880490
4,0.846868,0.846868,0.846868,8.471757e-01,8.471757e-01,8.471757e-01,0.846796,0.846796,0.846796,0.846784,0.846784,0.846784
...,...,...,...,...,...,...,...,...,...,...,...,...
252,NaN,NaN,NaN,6.682821e-12,6.682821e-12,6.682821e-12,NaN,NaN,NaN,NaN,NaN,NaN
253,NaN,NaN,NaN,6.691396e-12,6.691396e-12,6.691396e-12,NaN,NaN,NaN,NaN,NaN,NaN
254,NaN,NaN,NaN,6.692654e-12,6.692654e-12,6.692654e-12,NaN,NaN,NaN,NaN,NaN,NaN
255,NaN,NaN,NaN,6.692728e-12,6.692728e-12,6.692728e-12,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
run_names = list(residuals_df.columns)
for r in run_names:
    print(r)

primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.9-1.0) | residual_norms (median)
primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.9-1.0) | residual_norms (1st quartile)
primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.9-1.0) | residual_norms (3rd quartile)
primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.0) | residual_norms (median)
primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.0) | residual_norms (1st quartile)
primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.0) | residual_norms (3rd quartile)
primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.2) | residual_norms (median)
primal_random | csc | subsample | sketch_size = 158 

In [133]:
rep = {" (median)": "", " (1st quartile)": "", " (3rd quartile)": ""}
rep = dict((re.escape(k), v) for k, v in rep.items())
pattern = re.compile("|".join(rep.keys()))

run_names = {
    # col for col in self.residual_norms_df.columns
    pattern.sub(lambda m: rep[re.escape(m.group(0))], col) for col in residuals_df.columns
}
run_names = list(run_names)
run_names.sort()

print("\n~~~~ run_names: ")
for n in run_names:
    print(n)
print()


~~~~ run_names: 
primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.0) | residual_norms
primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.2) | residual_norms
primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.9-1.0) | residual_norms
primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = auto | accel_param = (0.0-0.0) | residual_norms



In [134]:
def format_run_names_exp(run_names):
    """
    Selects and formats run names based on run name, solver and sketch size
    """
    formatted_run_names = []
    for r in run_names:
        print(r)
        if "auto" in r:
            r = r"$\text{no acceleration}$"
        elif "accel_param = (0.8-1.0)" in r:
            r = r"$\text{acceleration }(\mu = 0.8, \nu = 1.0)$"
        elif "accel_param = (0.8-1.2)" in r:
            r = r"$\text{acceleration }(\mu = 0.8, \nu = 1.2)$"
        elif "accel_param = (0.9-1.0)" in r:
            r = r"$\text{acceleration }(\mu = 0.9, \nu = 1.0)$"
        else:
            r = r.split(" | ")[6].split(" ")[2]
            r = r.replace("(", r"($\mu = $")
            r = r.replace("-", r", $\nu = $")
            r = "acceleration " + r 
        formatted_run_names.append(r)
    return formatted_run_names

In [135]:
# format run names
formatted_run_names = format_run_names_exp(run_names)
print("~~~~ formatted_run_names: ")
for n in formatted_run_names:
    print(n)

primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.0) | residual_norms
primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.2) | residual_norms
primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.9-1.0) | residual_norms
primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = auto | accel_param = (0.0-0.0) | residual_norms
~~~~ formatted_run_names: 
$\text{acceleration }(\mu = 0.8, \nu = 1.0)$
$\text{acceleration }(\mu = 0.8, \nu = 1.2)$
$\text{acceleration }(\mu = 0.9, \nu = 1.0)$
$\text{no acceleration}$


# Plot

In [136]:
def plot_residual_v_iteration(run_name, formatted_run_name, residuals_df, fig):
    """
    Adds a single trace to the existing figure for experiment 9.2, 9.3 and 9.4
    """
    y = residuals_df[f"{run_name} (median)"]
    y = y[~np.isnan(y)]  # removing pading nans

    n_iterations = len(y)
    iterations = list(range(n_iterations))

    x = iterations.copy()

    y_upper = residuals_df[f"{run_name} (3rd quartile)"].values
    y_lower = residuals_df[f"{run_name} (1st quartile)"].values

    n_max_points = 25
    if n_iterations > n_max_points:
        step = int(n_iterations / n_max_points)
        iter_to_plot = iterations[::step]
        iter_to_plot.append(iterations[-1])
        iter_to_plot = list(dict.fromkeys(iter_to_plot))

        x_to_plot = [x[i] for i in iter_to_plot]
        y_to_plot = y[iter_to_plot]

        y_upper_to_plot = y_upper[iter_to_plot]
        y_lower_to_plot = y_lower[iter_to_plot]
    else:
        x_to_plot = x
        y_to_plot = y

        y_upper_to_plot = y_upper
        y_lower_to_plot = y_lower

    style_key = formatted_run_name.split(" | ")[0]

    current_color = COLOR_DICT[style_key]
    alpha = 0.2  # opacity of the error colored area
    current_color_rgba = ImageColor.getcolor(current_color, "RGB") + (alpha,)

    fig.add_trace(
        go.Scatter(
            x=x_to_plot,
            y=y_to_plot,
            name=formatted_run_name,
            mode="lines+markers",
            line=dict(color=current_color, dash=LINE_DICT[style_key]),
            marker=dict(symbol=SYMBOL_DICT[style_key], size=10),
        )
    )

    fig.add_trace(
        go.Scatter(
            x=x_to_plot,
            y=y_upper_to_plot,
            mode="lines",
            line=dict(width=0),
            marker=dict(color=f"rgba{current_color_rgba}"),
            showlegend=False,
        )
    )

    fig.add_trace(
        go.Scatter(
            x=x_to_plot,
            y=y_lower_to_plot,
            mode="lines",
            line=dict(width=0),
            marker=dict(color=f"rgba{current_color_rgba}"),
            fillcolor=f"rgba{current_color_rgba}",
            fill="tonexty",
            showlegend=False,
        )
    )

    return fig

In [159]:
@apply_style
def plot_runs_over_iterations(run_names, formatted_run_names, residuals_df):
    """Plots runs over iterations for experiment 9.2, 9.3 and 9.4"""
    fig = go.Figure()

    for run_name, formatted_run_name in zip(run_names, formatted_run_names):
        run_name.replace("residual_norms", "")
        if "direct" not in run_name:
            fig = plot_residual_v_iteration(
                run_name, formatted_run_name, residuals_df, fig
            )

    fig.update_layout(
        margin={"l": 20, "r": 20, "t": 20, "b": 20},
        xaxis_title="iteration",
        yaxis_title="relative residual norm",
        yaxis_type="log",
        # legend=dict(yanchor="bottom", y=0.05, xanchor="left", x=0.05),
        legend=dict(yanchor="top", y=0.95, xanchor="right", x=1.),
    )

    return fig

In [160]:
fig = plot_runs_over_iterations(run_names, formatted_run_names, residuals_df)
fig.show()

## Save

In [161]:
file_name = "acceleration_parameters_convergence_plot_iter.pdf"
path = "/home/nidham/phd/RidgeSketch/ridge_sketch/experiments/results/"
fig.write_image(os.path.join(path, file_name))

# Time plot

In [140]:
times_df = pd.read_csv("/home/nidham/phd/RidgeSketch/ridge_sketch/benchmark_results/accel_exp/times.csv", index_col=0)

df = times_df.copy()
df["error (1st quartile)"] = df["time (median)"] - df["time (1st quartile)"]
df["error (3rd quartile)"] = df["time (3rd quartile)"] - df["time (median)"]
df

,problem,sparse_format,solver,sketch_size,operator_mode,algo_mode,accel_param,time (median),time (1st quartile),time (3rd quartile),error (1st quartile),error (3rd quartile)
0,primal_random,csc,subsample,158,False,accel,"(0.9, 1.0)",2.954452,2.954452,2.954452,0.0,0.0
1,primal_random,csc,subsample,158,False,accel,"(0.8, 1.0)",2.087243,2.087243,2.087243,0.0,0.0
2,primal_random,csc,subsample,158,False,accel,"(0.8, 1.2)",2.416954,2.416954,2.416954,0.0,0.0
3,primal_random,csc,subsample,158,False,auto,"(0.0, 0.0)",1.318224,1.318224,1.318224,0.0,0.0


In [141]:
residuals_df

,primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.9-1.0) | residual_norms (median),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.9-1.0) | residual_norms (1st quartile),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.9-1.0) | residual_norms (3rd quartile),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.0) | residual_norms (median),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.0) | residual_norms (1st quartile),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.0) | residual_norms (3rd quartile),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.2) | residual_norms (median),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.2) | residual_norms (1st quartile),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.2) | residual_norms (3rd quartile),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = auto | accel_param = (0.0-0.0) | residual_norms (median),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = auto | accel_param = (0.0-0.0) | residual_norms (1st quartile),primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = auto | accel_param = (0.0-0.0) | residual_norms (3rd quartile)
0,1.000000,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.960681,0.960681,0.960681,9.606807e-01,9.606807e-01,9.606807e-01,0.960681,0.960681,0.960681,0.960681,0.960681,0.960681
2,0.918090,0.918090,0.918090,9.181836e-01,9.181836e-01,9.181836e-01,0.918067,0.918067,0.918067,0.918064,0.918064,0.918064
3,0.880549,0.880549,0.880549,8.807582e-01,8.807582e-01,8.807582e-01,0.880498,0.880498,0.880498,0.880490,0.880490,0.880490
4,0.846868,0.846868,0.846868,8.471757e-01,8.471757e-01,8.471757e-01,0.846796,0.846796,0.846796,0.846784,0.846784,0.846784
...,...,...,...,...,...,...,...,...,...,...,...,...
252,NaN,NaN,NaN,6.682821e-12,6.682821e-12,6.682821e-12,NaN,NaN,NaN,NaN,NaN,NaN
253,NaN,NaN,NaN,6.691396e-12,6.691396e-12,6.691396e-12,NaN,NaN,NaN,NaN,NaN,NaN
254,NaN,NaN,NaN,6.692654e-12,6.692654e-12,6.692654e-12,NaN,NaN,NaN,NaN,NaN,NaN
255,NaN,NaN,NaN,6.692728e-12,6.692728e-12,6.692728e-12,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
fig_time = go.Figure()

# (0.0, 0.0)
runtime = df[df["accel_param"]=="(0.0, 0.0)"]["time (median)"]
runtime = runtime.values[0]

y = residuals_df["primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = auto | accel_param = (0.0-0.0) | residual_norms (median)"]
y = y[~np.isnan(y)]
iterations = y.shape[0]
style_key = "$\\text{no acceleration}$"


fig_time.add_trace(
    go.Scatter(
        x=np.array(range(iterations)) * runtime / iterations,
        y=y.values,
        name=style_key,
        mode="lines+markers",
        line=dict(color=COLOR_DICT[style_key], dash=LINE_DICT[style_key]),
        marker=dict(symbol=SYMBOL_DICT[style_key], size=10),
    )
)


# (0.9, 1.0)
runtime = df[df["accel_param"]=="(0.9, 1.0)"]["time (median)"]
runtime = runtime.values[0]

y = residuals_df["primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.9-1.0) | residual_norms (median)"]
y = y[~np.isnan(y)]
iterations = y.shape[0]
style_key = "$\\text{acceleration }(\\mu = 0.9, \\nu = 1.0)$"


fig_time.add_trace(
    go.Scatter(
        x=np.array(range(iterations)) * runtime / iterations,
        y=y.values,
        name=style_key,
        mode="lines+markers",
        line=dict(color=COLOR_DICT[style_key], dash=LINE_DICT[style_key]),
        marker=dict(symbol=SYMBOL_DICT[style_key], size=10),
    )
)

# (0.8, 1.0)
runtime = df[df["accel_param"]=="(0.8, 1.0)"]["time (median)"]
runtime = runtime.values[0]

y = residuals_df["primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.0) | residual_norms (median)"]
y = y[~np.isnan(y)]
iterations = y.shape[0]
style_key = "$\\text{acceleration }(\\mu = 0.8, \\nu = 1.0)$"


fig_time.add_trace(
    go.Scatter(
        x=np.array(range(iterations)) * runtime / iterations,
        y=y.values,
        name=style_key,
        mode="lines+markers",
        line=dict(color=COLOR_DICT[style_key], dash=LINE_DICT[style_key]),
        marker=dict(symbol=SYMBOL_DICT[style_key], size=10),
    )
)

# (0.8, 1.2)
runtime = df[df["accel_param"]=="(0.8, 1.2)"]["time (median)"]
runtime = runtime.values[0]

y = residuals_df["primal_random | csc | subsample | sketch_size = 158 | no-op | algo_mode = accel | accel_param = (0.8-1.2) | residual_norms (median)"]
y = y[~np.isnan(y)]
iterations = y.shape[0]
style_key = "$\\text{acceleration }(\\mu = 0.8, \\nu = 1.2)$"


fig_time.add_trace(
    go.Scatter(
        x=np.array(range(iterations)) * runtime / iterations,
        y=y.values,
        name=style_key,
        mode="lines+markers",
        line=dict(color=COLOR_DICT[style_key], dash=LINE_DICT[style_key]),
        marker=dict(symbol=SYMBOL_DICT[style_key], size=10),
    )
)

In [151]:
fig_time.update_layout(
    margin={"l": 20, "r": 20, "t": 20, "b": 20},
    xaxis_title="time (s)",
    yaxis_title="relative residual norm",
    yaxis_type="log",
    legend=dict(yanchor="top", y=1., xanchor="right", x=1.),
    template="plotly_white", 
    font=dict(size=20,)
)

In [152]:
file_name = "acceleration_parameters_convergence_plot_time.pdf"
path = "/home/nidham/phd/RidgeSketch/ridge_sketch/experiments/results/"
fig_time.write_image(os.path.join(path, file_name))